In [ ]:
# Created by GB - 10/22
# Google Colab was used in this code
import numpy as np
import pandas as pd
import PIL
import pathlib
import os
import random
import matplotlib.pyplot as plt
import warnings

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path1 = r'/content/drive/MyDrive/Colab Notebooks/Images/iron man comics_dir_resized'
path2 = r'/content/drive/MyDrive/Colab Notebooks/Images/spider man comics_dir_resized'

Mounted at /content/drive/


In [ ]:
    ## Preparing the data ##

# Convert images into numpy arrays #
# Concatenate the arrays #
# Shuffle #
# Split into train and test sets #

In [ ]:
# Converting images to numpy arrays 
def numpy_arrays(path1,path2):
    delimiter = chr(92)
    path_list = []
    arrays_list1 = []
    arrays_list2 = []
    path_list.append(path1)
    path_list.append(path2)
    counter = 0
    for paths in range(len(path_list)):
        listdir = os.listdir(path_list[paths])
        counter += 1
        for items in listdir:
            items.split()
            if counter == 1:
                image = PIL.Image.open(path1+'/'+items) #problem with delimiter !WARNING!
            else:
                image = PIL.Image.open(path2+'/'+items)
            image_inf = image.getdata()
            sub_array = np.array(image_inf)
            if counter == 1:
                arrays_list1.append(sub_array)
            else:
                arrays_list2.append(sub_array)
    
    ones= np.ones((len(arrays_list1),), dtype=int)
    ones = ones.tolist()
    zeros= np.zeros((len(arrays_list2),), dtype=int)
    zeros = zeros.tolist()
    return arrays_list1,arrays_list2,ones,zeros

In [ ]:
def conc_lists(array1,array2,ones,zeros):
    arrayX = array1
    arrayY = ones
    for i in range(len(array2)):
        arrayX.append(array2[i])
    for x in range(len(zeros)):
        arrayY.append(zeros[x])
        
    return arrayX, arrayY

In [ ]:
def shuffle(X1,X2):
    X3 = list(zip(X1,X2))
    random.shuffle(X3)
    X1,X2 = zip(*X3)
    return X1,X2

In [ ]:
def train_test_split(array1,array2,train_size):
    len_array1 = len(array1)
    len_array2 = len(array2)
    array1_train_size = int(train_size*len_array1)
    array2_train_size = int(train_size*len_array2)
    X_train = array1[:array1_train_size]
    Y_train = array2[:array1_train_size]
    X_test = array1[array2_train_size:]
    Y_test = array2[array2_train_size:]
    return X_train,Y_train,X_test,Y_test

In [ ]:
array1,array2,ones,zeros=numpy_arrays(path1,path2)
print('class 1 len: ',len(array1))
print('class 1 data type: ',type(array1))
print('class 1 image size (multiplicated): ',array1[1].shape)
print('class 2 len: ',len(array2))
print('class 2 data type: ',type(array2))
print('class 2 image size (multiplicated): ',array2[1].shape)
print('class 1 y len: ',len(ones))

class 1 len:  656
class 1 data type:  <class 'list'>
class 1 image size (multiplicated):  (160000,)
class 2 len:  656
class 2 data type:  <class 'list'>
class 2 image size (multiplicated):  (160000,)
class 1 y len:  656


In [ ]:
# This part is used to concancate class arrays and their y values
arrayX,arrayY = conc_lists(array1,array2,ones,zeros)
len(arrayY)

1312

In [ ]:
# This part is used to shuffle all data with same way.
# After running, we have more realistic data to classificate
array_X,array_Y = shuffle(arrayX,arrayY)
len(array_X)

1312

In [ ]:
# This part is to split the data into train and test datas.
train_X,train_Y,test_X,test_Y=train_test_split(array_X,array_Y,train_size=0.9)
print('length of X values of train data: ',len(train_X))
print('length of Y values of train data: ',len(train_Y))
print('length of X values of test data: ',len(test_X))
print('length of Y values of test data: ',len(test_Y))
train_X = np.array(train_X)
train_Y = np.array(train_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)

length of X values of train data:  1180
length of Y values of train data:  1180
length of X values of test data:  132
length of Y values of test data:  132


In [ ]:
# Normalizing the datas #
standard_X_train =  (train_X - np.average(train_X)) / (np.std(train_X))
standard_X_test =  (test_X - np.average(test_X)) / (np.std(test_X))

In [ ]:
### Classification with Logistic Regression ###

In [ ]:
# Sigmoid function for logistic regression #
def sigmoid_function(x):
    return 1 / (1 + np.exp(-1*x))

In [ ]:
def loss_gradient(x,y,sig_y):
    return np.dot(x.T,(sig_y-y))

In [ ]:
# Defining cross entropy loss #
def cross_entropy(y,t):
    return -1 * (t * np.log(y) + (1 - t) * np.log(1 - y))

In [ ]:
def accuracy_s(y_true,y_pred):
    len_of_t = len(y_true)
    len_of_p = len(y_pred)
    true = 0
    if len_of_t != len_of_p:
        print("The sizes are not matched !")
    else:
        for _ in range(len_of_t):
            if y_true[_]==y_pred[_]:
                true += 1
    
    accuracy = true/len_of_t
    return accuracy

In [ ]:
# Parameter initialization #
param_init = {}
param_init["w"] = np.random.randn(train_X.shape[1])
param_init["b"] = 0.05

In [ ]:
param_init["w"].shape

(160000,)

In [ ]:
# Logistic regression parameter optimization #
def logreg(x,y,parameters,n_iterations,lrate):
    lenx = len(x)
    w = parameters["w"]
    b = parameters["b"]
    loss_list = []

    for _ in range(n_iterations):
        sig_y = sigmoid_function(np.dot(x,w)+b)
        cross_loss = cross_entropy(sig_y,y)
        loss_list.append(cross_loss)
        dW = loss_gradient(x,y,sig_y) / lenx
        db = np.sum(sig_y-y) / lenx
        new_w = w - lrate * dW
        new_b = b - lrate * db
    
    parameters["w"] = new_w
    parameters["b"] = new_b
    return parameters,loss_list

In [ ]:
# Defining parameters and training the model #
warnings.filterwarnings('ignore')
lrate = 0.001
n_iterations = 1000
parameters_obtained,loss = logreg(standard_X_train,train_Y,param_init,n_iterations,lrate)

In [ ]:
# Accuracy with test data
new_test_X = np.dot(standard_X_test, parameters_obtained["w"] ) + parameters_obtained["b"] 
new_test_y = sigmoid_function(new_test_X) > 1/2  
new_test_y = [int(y) for y in new_test_y]


In [ ]:
accuracy = accuracy_s(test_Y,new_test_y)
accuracy

0.5227272727272727

In [ ]:
# Sklearn Example

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logregsk = LogisticRegression(max_iter=3000)
logregsk.fit(standard_X_train,train_Y)
pred_logreg = logregsk.predict(standard_X_test)
accuracy_sk = accuracy_score(test_Y, pred_logreg)
accuracy_sk

0.5303030303030303

In [ ]:
parameters_obtained

{'w': array([-0.05846678,  2.46380384,  0.56637152, ...,  0.86249728,
         0.09738942,  0.62857064]), 'b': 0.049937821917104046}

In [ ]:
# Tuning
n_iterations_list = [300, 400, 500, 600, 700]
lrate_list = [0.001, 0.01, 0.1, 0.5, 1]
accuracy_list = []
Tuning_list = []
loss_list = []
for k in n_iterations_list:
    for i in lrate_list:
        parameters_obtained,loss = logreg(standard_X_train,train_Y,param_init,k,i)
        new_test_X = np.dot(standard_X_test, parameters_obtained["w"] ) + parameters_obtained["b"] 
        new_test_y = sigmoid_function(new_test_X) > 1/2  
        new_test_y = [int(y) for y in new_test_y]
        accuracy = accuracy_s(test_Y,new_test_y)
        accuracy_list.append(accuracy)
        arb_list = [k,i,accuracy]
        Tuning_list.append(arb_list)
        loss_list.append(loss)